# Lab | Data Cleaning and Formatting

Objective: Apply Python programming to perform data cleaning and formatting, including identifying and handling with missing values, duplicates, outliers, errors, and using various formatting techniques such as changing data types, renaming columns, and handling string values, to prepare data for analysis.

In this lab, we will be working with the customer data from an insurance company, which can be found in the CSV file located at the following link: https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv


In [1]:
# Your code here

#importing pandas 
import pandas as pd

# read csv file from the given URL and save a csv file in current folder
df = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv')
df.to_csv('costumer.csv', sep=';', encoding='utf-8', index=False)


In [2]:
def showUnique(data):
    for row in data:
        print('values of column:',row)
        l = 0
        print(data[row].unique())
        l = len(data[row].unique())
        print('number of unique values:', l, '\n')
        
def nullTable(dataset):
    round(dataset.isna().sum()/len(dataset),4)*100  # shows the percentage of null values in a column
    nulls_df = pd.DataFrame(round(dataset.isna().sum()/len(dataset),4)*100)
    nulls_df
    nulls_df = nulls_df.reset_index()
    nulls_df
    nulls_df.columns = ['header_name', 'percent_nulls']
    display(nulls_df)
    display(dataset.isna().sum())

# Challenge 1: Data Cleaning and Formatting

## Exercise 1: Cleaning Column Names

To ensure consistency and ease of use, standardize the column names of the dataframe. Start by taking a first look at the dataframe and identifying any column names that need to be modified. Use appropriate naming conventions and make sure that column names are descriptive and informative.

In [3]:
print(df.columns)

# data2['customer lifetime value'] = data2['customer lifetime value'].str.replace('%', '')
# Reused some functions from previous labs and combined them into a new columnCleaning function


def columnCleaning(dataset):
    dataset.columns= [col.lower() for col in dataset.columns]
    dataset.columns= [col.replace(' ', '_') for col in dataset.columns]
    dataset.rename(columns={'st':'state'}, inplace=True)
    dataset.rename(columns={'monthly_premium_auto':'monthly_premium_costs'}, inplace=True)
    
    
columnCleaning(df)
print(df.columns)
    

Index(['Customer', 'ST', 'GENDER', 'Education', 'Customer Lifetime Value',
       'Income', 'Monthly Premium Auto', 'Number of Open Complaints',
       'Policy Type', 'Vehicle Class', 'Total Claim Amount'],
      dtype='object')
Index(['customer', 'state', 'gender', 'education', 'customer_lifetime_value',
       'income', 'monthly_premium_costs', 'number_of_open_complaints',
       'policy_type', 'vehicle_class', 'total_claim_amount'],
      dtype='object')


In [4]:

# Using null table function from previous lab to check the data 
# do the same cleaning as previous lab 
# Exercise 4




nullTable(df)

# creating a backup
df_backup = df.copy()

# droping all rows with null values because their are now infos in the rows which I can use for further calculation

df = df.dropna(subset=['customer'])
nullTable(df)

,header_name,percent_nulls
0,customer,73.28
1,state,73.28
2,gender,76.20
3,education,73.28
4,customer_lifetime_value,73.35
5,income,73.28
6,monthly_premium_costs,73.28
7,number_of_open_complaints,73.28
8,policy_type,73.28
9,vehicle_class,73.28


customer                     2937
state                        2937
gender                       3054
education                    2937
customer_lifetime_value      2940
income                       2937
monthly_premium_costs        2937
number_of_open_complaints    2937
policy_type                  2937
vehicle_class                2937
total_claim_amount           2937
dtype: int64

,header_name,percent_nulls
0,customer,0.00
1,state,0.00
2,gender,10.92
3,education,0.00
4,customer_lifetime_value,0.28
5,income,0.00
6,monthly_premium_costs,0.00
7,number_of_open_complaints,0.00
8,policy_type,0.00
9,vehicle_class,0.00


customer                       0
state                          0
gender                       117
education                      0
customer_lifetime_value        3
income                         0
monthly_premium_costs          0
number_of_open_complaints      0
policy_type                    0
vehicle_class                  0
total_claim_amount             0
dtype: int64

## Exercise 2: Cleaning invalid Values

The dataset contains columns with inconsistent and incorrect values that could affect the accuracy of our analysis. Therefore, we need to clean these columns to ensure that they contain only valid data. For example, the gender column contains various inconsistent values such as "F", "M", "Femal", "Male", "female", which need to be standardized.

Note that this exercise will focus only on cleaning inconsistent values and will not involve handling null values (NaN or None).

In [5]:
# Your code here
# Using Code from previous lab and creating cleaningGenderColumn function

def cleaningGender(x):
    if x in ['M', 'MALE']:
        return 'Male'
    elif x.startswith('F'):
        return 'Female'
    else:
        x
        
def cleaningState(x):
    if x in ['WA']:
        return 'Washington'
    elif x in ['Cali']:
        return 'California'
    elif x in ['AZ']:
        return 'Arizona'
    else: 
        return x


def cleaningValues(dataframe):
    # Exercise 4: fill the missing gender values -> destribution between men and female was not so far away from each other and using all null values with only one type of them (men | female ) had a big impact of distribution 
    
    # checking code
    # print(df['gender'].unique())
    # genderCounts = df['gender'].value_counts(dropna=False)
    # print(genderCounts) 
    
    # CLEANING GENDER COLUMN
    dataframe['gender']=df['gender'].fillna(method='bfill')
    dataframe['gender'] = list(map(str.upper, df['gender']))
    dataframe['gender'] = list(map(cleaningGender, dataframe['gender']))
    
    #checking code
    # print(df['gender'].unique())
    # print(genderCounts)

    # CLEANING STATE COLUMN
    dataframe['state'] = list(map(cleaningState, dataframe['state']))
    
    # CLEAINING EDUCATION COLUMN
    dataframe['education'] = dataframe['education'].str.replace('Bachelors', 'Bachelor')
    dataframe['education'].unique()
    
    # CLEANING CUSTOMER LIFETIME VALUE COLUMN
    # Using median to fill null values
    dataframe['customer_lifetime_value'] = dataframe['customer_lifetime_value'].str.replace('%', '')
    dataframe['customer_lifetime_value'] =  pd.to_numeric(dataframe['customer_lifetime_value'], errors='coerce')

    customer_lifetime_median = dataframe['customer_lifetime_value'].median()
    dataframe['customer_lifetime_value'] = dataframe['customer_lifetime_value'].fillna(customer_lifetime_median)
    
    # SPLITING NUMBER OF OPEN COMPLAINTS COLUMN
    # Assuming that the 3 different values [x/x/xx] representing different status of tickets or something like this e.g. [number of closed tickets/number of tickets in progress/open tickets] or different types of complaints 
    dataframe[['complaint_type1', 'complaint_type2', 'complaint_type3']] = dataframe['number_of_open_complaints'].str.split('/', expand=True)

    dataframe['complaint_type1'] =  pd.to_numeric(dataframe['complaint_type1'], errors='coerce')
    dataframe['complaint_type2'] =  pd.to_numeric(dataframe['complaint_type2'], errors='coerce')
    dataframe['complaint_type3'] =  pd.to_numeric(dataframe['complaint_type3'], errors='coerce')
    
    # CLEANING INCOME COLUMN
    
    income_median= dataframe['income'].median()
    dataframe['income'] = dataframe['income'].replace(0, income_median)

    dataframe['income'].value_counts()
            
    
cleaningValues(df)

showUnique(df)

    
    

values of column: customer
['RB50392' 'QZ44356' 'AI49188' ... 'GS98873' 'CW49887' 'MY31220']
number of unique values: 1071 

values of column: state
['Washington' 'Arizona' 'Nevada' 'California' 'Oregon']
number of unique values: 5 

values of column: gender
['Female' 'Male']
number of unique values: 2 

values of column: education
['Master' 'Bachelor' 'High School or Below' 'College' 'Doctor']
number of unique values: 5 

values of column: customer_lifetime_value
[ 588174.235  697953.59  1288743.17  ... 2031499.76   323912.47
  899704.02 ]
number of unique values: 1028 

values of column: income
[36234. 48767. 36357. 62902. 55350. 14072. 28812. 77026. 99845. 83689.
 24599. 25049. 28855. 51148. 66140. 57749. 61040. 17870. 97541. 29723.
 86584. 75690. 23158. 65999. 37260. 68987. 42305. 65706. 53243. 50071.
 60021. 43244. 46896. 10105. 23218. 24804. 33190. 36014. 67163. 16701.
 46623. 64749. 16969. 11621. 17622. 11489. 66525. 33663. 22547. 61486.
 29879. 93011. 65186. 26840. 75644. 38984

## Exercise 3: Formatting data types

The data types of many columns in the dataset appear to be incorrect. This could impact the accuracy of our analysis. To ensure accurate analysis, we need to correct the data types of these columns. Please update the data types of the columns as appropriate. 

It is important to note that this exercise does not involve handling null values (NaN or None).

In [6]:
# Your code here
# allready handled in the previous exercises 
df.dtypes

customer                      object
state                         object
gender                        object
education                     object
customer_lifetime_value      float64
income                       float64
monthly_premium_costs        float64
number_of_open_complaints     object
policy_type                   object
vehicle_class                 object
total_claim_amount           float64
complaint_type1                int64
complaint_type2                int64
complaint_type3                int64
dtype: object

## Exercise 4: Dealing with Null values

Identify any columns with null or missing values. Identify how many null values each column has. You can use the isnull() function in pandas to find columns with null values.

Decide on a strategy for handling the null values. There are several options, including:

- Drop the rows or columns with null values
- Fill the null values with a specific value (such as the column mean or median for numerical variables and mode for categorical variables)
- Fill the null values with the previous or next value in the column
- Fill the null values based on a more complex algorithm or model (we haven't studied this yet)

Implement your chosen strategy to handle the null values. You can use the fillna() function in pandas to fill null values or dropna() function to drop null values.

Verify that your strategy has successfully handled the null values. You can use the isnull() function again to check if there are still null values in the dataset.

Remember to document your process and explain your reasoning for choosing a particular strategy for handling null values. 


After formatting data types, as a last step, convert all the numeric variables to integers using applymap.

Save the cleaned dataset to a new CSV file.

In [7]:
# Your code here


# save a new csv
df.to_csv('customerCleaned.csv', sep=',', encoding='utf-8', index=False)


## Exercise 5: Dealing with duplicates

Use the .duplicated() method to identify any duplicate rows in the dataframe.

Decide on a strategy for handling the duplicates. Options include:
- Dropping all duplicate rows
- Keeping only the first occurrence of each duplicated row
- Keeping only the last occurrence of each duplicated row
- Dropping duplicates based on a subset of columns
- Dropping duplicates based on a specific column

Implement your chosen strategy using the drop_duplicates() function.

Verify that your strategy has successfully handled the duplicates by checking for duplicates again using .duplicated().

Remember to document your process and explain your reasoning for choosing a particular strategy for handling duplicates.

Save the cleaned dataset to a new CSV file.

In [8]:
# Your code here

df.duplicated()

# no duplicates found 

0       False
1       False
2       False
3       False
4       False
        ...  
1066    False
1067    False
1068    False
1069    False
1070    False
Length: 1071, dtype: bool

# Challenge 2: creating functions on a separate `py` file

Put all the data cleaning and formatting steps into functions, and create a main function that performs all the cleaning and formatting. 

Write these functions in a separate .py file(s). By putting these steps into functions, we can make the code more modular and easier to maintain. 

*Hint: autoreload module is a utility module in Python that allows you to automatically reload modules in the current session when changes are made to the source code. This can be useful in situations where you are actively developing code and want to see the effects of changes you make without having to constantly restart the Python interpreter or Jupyter Notebook kernel.*

In [11]:
# Your code here
# allready handled in the previous exercises 
functionTest = pd.read_csv('https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv')
functionTest.to_csv('costumerFunctionTest.csv', sep=';', encoding='utf-8', index=False)

import cleaning as cl 

display(functionTest)
functionTest = cl.clean(functionTest)
display(functionTest)


# save a new csv
functionTest.to_csv('customerCleanedFunctionTest.csv', sep=',', encoding='utf-8', index=False)

,Customer,ST,GENDER,Education,Customer Lifetime Value,Income,Monthly Premium Auto,Number of Open Complaints,Policy Type,Vehicle Class,Total Claim Amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,QZ44356,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,AI49188,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,WW63253,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,GA49547,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...,...
4003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


test1
Starting cleaning Null Values
Null cleaning complete
Starting cleaning gender column
Gender column cleaning complete
Starting cleaning state column
state column cleaning complete


c:\Users\Domen\Master_IronHack\00_IH-Labs\lab-dw-data-cleaning-and-formatting\cleaning.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['gender'] = dataframe['gender'].fillna(method='bfill')
c:\Users\Domen\Master_IronHack\00_IH-Labs\lab-dw-data-cleaning-and-formatting\cleaning.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['gender'] = dataframe['gender'].str.upper()
c:\Users\Domen\Master_IronHack\00_IH-Labs\lab-dw-data-cleaning-and-formatting\cleaning.py:50: SettingWithCopyWarn

,customer,state,gender,education,customer_lifetime_value,income,monthly_premium_costs,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount,complaint_type1,complaint_type2,complaint_type3
0,RB50392,Washington,Female,Master,588174.235,36234.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934,1,0,0
1,QZ44356,Arizona,Female,Bachelor,697953.590,36234.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935,1,0,0
2,AI49188,Nevada,Female,Bachelor,1288743.170,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247,1,0,0
3,WW63253,California,Male,Bachelor,764586.180,36234.0,106.0,1/0/00,Corporate Auto,SUV,529.881344,1,0,0
4,GA49547,Washington,Male,High School or Below,536307.650,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,TM65736,Oregon,Male,Master,305955.030,38644.0,78.0,1/1/00,Personal Auto,Four-Door Car,361.455219,1,1,0
1067,VJ51327,California,Female,High School or Below,2031499.760,63209.0,102.0,1/2/00,Personal Auto,SUV,207.320041,1,2,0
1068,GS98873,Arizona,Female,Bachelor,323912.470,16061.0,88.0,1/0/00,Personal Auto,Four-Door Car,633.600000,1,0,0
1069,CW49887,California,Female,Master,462680.110,79487.0,114.0,1/0/00,Special Auto,SUV,547.200000,1,0,0


# Challenge 3: Analyzing Clean and Formated Data

You have been tasked with analyzing the data to identify potential areas for improving customer retention and profitability. Your goal is to identify customers with a high policy claim amount and a low customer lifetime value.

In Pandas Lab, we only looked at high policy claim amounts because we couldn't look into low customer lifetime values. If we tried to do work with that column, we wouldn't have been able to do it because customer lifetime value wasn't clean and in its proper format. So after cleaning and formatting the data, lets get some more interesting insights!

Instructions:

- Review again the statistics for total claim amount and customer lifetime value to gain an understanding of the data.
- To identify potential areas for improving customer retention and profitability, we want to focus on customers with a high policy claim amount and a low customer lifetime value. Consider customers with a high policy claim amount to be those in the top 25% of the total claim amount, and clients with a low customer lifetime value to be those in the bottom 25% of the customer lifetime value. Create a pandas DataFrame object that contains information about customers with a policy claim amount greater than the 75th percentile and a customer lifetime value in the bottom 25th percentile.
- Use DataFrame methods to calculate summary statistics about the high policy claim amount and low customer lifetime value data. To do so, select both columns of the dataframe simultaneously and pass it to the .describe() method. This will give you descriptive statistics, such as mean, median, standard deviation, minimum and maximum values for both columns at the same time, allowing you to compare and analyze their characteristics.

In [10]:
# Your code here